In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import requests
import folium

In [ ]:
# Cargar archivo csv
dataset = pd.read_csv("SCRAPER.csv")

In [2]:
# Quitar filas sin precio
dataset = dataset[dataset['precio'].notnull()]

In [ ]:
# Agregar columna con diferenciación entre UF y $
if dataset['precio'].str.contains('UF').any():
    dataset['moneda'] = dataset['precio'].apply(lambda x: 'UF' if 'UF' in x else '$')

# Transformar precio a valor numérico
def extraer_precio(texto):
    """Extraer precio de una propiedad"""
    patrones_precio = [
        r'\$\s*\d{1,3}(?:\.\d{3})*(?:,\d+)?',
        r'\d{1,3}(?:\.\d{3})*(?:,\d+)?\s*UF',
        r'UF\s*\d{1,3}(?:\.\d{3})*(?:,\d+)?',
        r'\d{1,3}(?:\.\d{3})*(?:,\d+)?\s*CLP',
        r'CLP\s*\d{1,3}(?:\.\d{3})*(?:,\d+)?'
    ]
    
    for patron in patrones_precio:
        match = re.search(patron, texto, re.IGNORECASE)
        if match:
            precio = match.group().strip()
            # Normalizar espacios
            precio = re.sub(r'\s+', ' ', precio)
            return precio
    
    return None

def precio_a_float(precio_str):
    """Convierte un precio extraído a float"""
    if precio_str is None:
        return None
    
    # Eliminar símbolos de $ y UF/CLP
    limpio = re.sub(r'[^\d,.-]', '', precio_str)

    # Quitar puntos de miles y cambiar coma por punto
    limpio = limpio.replace(".", "").replace(",", ".")
    
    try:
        return float(limpio)
    except ValueError:
        return None

dataset['precio_valor'] = dataset['precio'].apply(extraer_precio)
dataset['precio_valor'] = dataset['precio'].apply(precio_a_float)

In [ ]:
#API Key
api_key = "---------------------------" # N° de API Key (Privado)

# Endpoint CMF para obtener UF en formato JSON
url = "https://api.cmfchile.cl/api-sbifv3/recursos_api/uf"
params = {
    "apikey": api_key,
    "formato": "json"
}

# Hacemos la consulta
response = requests.get(url, params=params)
response.raise_for_status()

# Convertimos la respuesta en JSON
data = response.json()

# Extraemos el valor de la UF
uf_entry = data.get("UFs", [None])[0]

if uf_entry and "Valor" in uf_entry:
    uf_valor_str = uf_entry["Valor"]
    fecha = uf_entry.get("Fecha", "Desconocida")
    
    # Convertimos de string a float
    uf_valor_float = float(uf_valor_str.replace(".", "").replace(",", "."))
    
    print(f"Valor UF del {fecha}: {uf_valor_float}")
else:
    print("No se encontró el valor de la UF para la consulta.")

Valor UF del 2025-09-08: 39474.24


In [ ]:
# Columna valor en pesos 
dataset['valor_en_pesos'] = np.where(
    dataset['moneda'].str.contains("UF", case=False, na=False),
    round(dataset['precio_valor'] * uf_valor_float,0),
    dataset['precio_valor']
)

# Columna valor en UF
dataset['valor_en_uf'] = np.where(
    dataset['moneda'].str.contains("\$", case=False, na=False),
    round(dataset['precio_valor'] / uf_valor_float, 2),
    dataset['precio_valor']
)

In [6]:
dataset

,titulo,precio,ubicacion,dormitorios,baños,estacionamientos,m2_construccion,m2_totales,m2_desde,tipo_propiedad,...,fecha_publicacion,precio_m2_construccion,precio_m2_terreno,direccion_exacta,costos_mantenimiento,descripcion_completa,moneda,precio_valor,valor_en_pesos,valor_en_uf
0,Inmobiliaria GPR Puerto Montt Condominio Bosqu...,"UF2.632,00",Puerto Montt,2.0,2.0,NaN,52.0,NaN,NaN,NaN,...,NaN,NaN,NaN,¡Pregunta al anunciante!,NaN,NaN,UF,2632.0,1.038962e+08,2632.00
1,Inmobiliaria GPR Puerto Montt Condominio Hacie...,"UF3.929,00",Puerto Montt,3.0,3.0,NaN,72.0,NaN,NaN,NaN,...,NaN,NaN,NaN,¡Pregunta al anunciante!,NaN,NaN,UF,3929.0,1.550943e+08,3929.00
2,Inmobiliaria Aconcagua Puerto Montt Condominio...,"UF2.750,00",Puerto Montt,2.0,1.0,NaN,50.0,NaN,NaN,NaN,...,NaN,NaN,NaN,¡Pregunta al anunciante!,NaN,NaN,UF,2750.0,1.085542e+08,2750.00
3,Valentina Vallarino Sulzer Puerto Montt CASA C...,"UF2.850,00",Puerto Montt,3.0,2.0,NaN,130.0,NaN,NaN,casa,...,27/08/2025,"UF21,92",NaN,¡Pregunta al anunciante!,NaN,Puerto Montt\n27/08/2025\nAnterior\n7/842\nSig...,UF,2850.0,1.125016e+08,2850.00
4,TERRAGESTION Gestión Inmobiliaria Puerto Montt...,$370.000.000,Puerto Montt,3.0,3.0,2.0,140.0,NaN,NaN,departamento,...,07/08/2025,$2.642.857,NaN,¡Pregunta al anunciante!,NaN,Puerto Montt\n07/08/2025\nAnterior\n8/842\nSig...,$,370000000.0,3.700000e+08,9373.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
833,Cruz Balmaceda Propiedades - Constanza Osses P...,$45.000.000,Puerto Montt,NaN,NaN,NaN,NaN,NaN,NaN,parcela,...,18/09/2024,NaN,$8.973,NaN,NaN,Resalta este anuncio\nPuerto Montt\n18/09/2024...,$,45000000.0,4.500000e+07,1139.98
834,Cruz Balmaceda Propiedades - Francisco Puerto ...,"UF36.000,00",Puerto Montt,6.0,3.0,NaN,NaN,NaN,NaN,terreno,...,30/01/2025,NaN,"UF6,00",¡Pregunta al anunciante!,NaN,Resalta este anuncio\nPuerto Montt\n30/01/2025...,UF,36000.0,1.421073e+09,36000.00
835,ASESORIA Y GESTION SURPROP SPA Puerto Montt Op...,$189.900.000,Puerto Montt,4.0,4.0,1.0,136.0,NaN,NaN,terreno,...,19/06/2025,$1.396.323,$759.600,¡Pregunta al anunciante!,NaN,Resalta este anuncio\nPuerto Montt\n19/06/2025...,$,189900000.0,1.899000e+08,4810.73
836,María Ester Larrañaga Puerto Montt Casa excele...,"UF7.000,00",Puerto Montt,3.0,2.0,1.0,115.0,NaN,NaN,casa,...,13/07/2025,"UF60,86","UF20,00",¡Pregunta al anunciante!,NaN,Resalta este anuncio\nPuerto Montt\n13/07/2025...,UF,7000.0,2.763197e+08,7000.00


In [ ]:
# Función para separar las coordenadas
def separar_coordenadas(coord_str):
    """
    Separa una cadena de coordenadas en latitud y longitud
    """
    if pd.isna(coord_str):
        return pd.Series([np.nan, np.nan])
    
    try:
        # Limpiar la cadena y dividir por la coma
        coords = str(coord_str).strip().split(',')
        if len(coords) == 2:
            lat = float(coords[0].strip())
            lon = float(coords[1].strip())
            return pd.Series([lat, lon])
        else:
            return pd.Series([np.nan, np.nan])
    except:
        return pd.Series([np.nan, np.nan])

# Aplicar la función para separar las coordenadas
dataset[['latitud', 'longitud']] = dataset['coordenadas'].apply(separar_coordenadas)

# Eliminar filas con valores NaN en las coordenadas
dataset_clean = dataset.dropna(subset=['latitud', 'longitud'])

In [ ]:
# Calcular el centro del mapa
center_lat = dataset_clean['latitud'].mean()
center_lon = dataset_clean['longitud'].mean()

# Crear el mapa base
mapa = folium.Map(
    location=[center_lat, center_lon], 
    zoom_start=11,
    tiles='OpenStreetMap'
)

# Agregar Font
font_awesome_css = """
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css">
"""
mapa.get_root().html.add_child(folium.Element(font_awesome_css))

# Función para formatear la información del popup
def crear_popup_info(row):
    """Crea el contenido HTML para el popup"""
    
    # Formatear tipo de propiedad
    tipo = row['tipo_propiedad'].title() if pd.notna(row['tipo_propiedad']) else 'No especificado'


    # Formatear enlace
    enlace = f"<a href='{row['enlace']}' target='_blank' style='color: #2E86AB;'>🔗 Ver más</a>" if pd.notna(row['enlace']) else 'No disponible'

    # Formatear valor en UF
    if pd.notna(row['valor_en_uf']):
        valor = f"{row['valor_en_uf']:,.2f} UF"
        # Convertir a pesos chilenos aproximados
        valor_clp = f"≈ ${row['valor_en_uf'] * uf_valor_float:,.0f} CLP"
    else:
        valor = "No especificado"
        valor_clp = ""
    
    # Formatear m2 de construcción
    m2_construccion = f"{row['m2_construccion']:,.0f} m²" if pd.notna(row['m2_construccion']) else 'No especificado'
    
    # Agregar m2 de terreno
    m2_terreno = ""
    if 'm2_terreno' in row.index and pd.notna(row['m2_terreno']):
        m2_terreno = f"<br><b>Terreno:</b> {row['m2_terreno']:,.0f} m²"
    
    # Formatear dormitorios
    dormitorios = f"{int(row['dormitorios'])} dormitorios" if 'dormitorios' in row.index and pd.notna(row['dormitorios']) else ''

    # Formatear baños
    baños = f"{int(row['baños'])} baños" if 'baños' in row.index and pd.notna(row['baños']) else ''
    
    # fecha de publicación
    fecha_pub = f"{row['fecha_publicacion']}" if 'fecha_publicacion' in row.index and pd.notna(row['fecha_publicacion']) else ''
    

    # Crear el HTML del popup
    popup_html = f"""
    <div style="font-family: Arial; font-size: 12px; width: 220px;">
        <h4 style="margin: 0 0 10px 0; color: #2E86AB; border-bottom: 1px solid #ddd;">
            <i class="fa {iconos_tipo.get(tipo.lower(), 'fa-question')}"></i> {tipo}
        </h4>
        <p style="margin: 5px 0;">
            <b>💰 Valor:</b> {valor}<br>
            <span style="color: #666;">{valor_clp}</span>
        </p>
        <p style="margin: 5px 0;">
            <b>📐 Construcción:</b> {m2_construccion}{m2_terreno}
        </p>
        <p style="margin: 5px 0;">
            <b>🛏️ Dormitorios:</b> {dormitorios}
        </p>
        <p style="margin: 5px 0;">
            <b>🛁 Baños:</b> {baños}
        </p>
        <p style="margin: 5px 0;">
            <b>📅 Fecha de Publicación:</b> {fecha_pub}
        </p>
        <p style="margin: 5px 0; font-size: 11px;">
            {enlace}
        </p>
    </div>
    """
    
    return popup_html

# Definir íconos según tipo de propiedad
iconos_tipo = {
    'casa': 'fa-home',
    'departamento': 'fa-building', 
    'parcela': 'fa-tree',
    'oficina': 'fa-briefcase',
    'local': 'fa-shopping-cart',
    'terreno': 'fa-map',
    'bodega': 'fa-archive',
    'otro': 'fa-question'
}

# Función para determinar color
def obtener_color_por_valor(valor):
    """Determina el color según el valor de la propiedad con rangos fijos"""
    if pd.isna(valor):
        return '#808080'  # Si no tiene valor

    # Rangos valor en UF
    if valor <= 1000:
        return '#00B050'
    elif valor <= 2500:
        return '#92D050'
    elif valor <= 5000:
        return '#FF8C00'
    elif valor <= 10000:
        return '#FF4500'
    else:
        return '#B22222'

# Agregar los puntos como Markers con íconos
for idx, row in dataset_clean.iterrows():
    if pd.notna(row['latitud']) and pd.notna(row['longitud']):
        
        # Determinar ícono según tipo de propiedad
        tipo_lower = str(row['tipo_propiedad']).lower() if pd.notna(row['tipo_propiedad']) else 'otro'
        icono = iconos_tipo.get(tipo_lower, 'question-sign')
        
        # Determinar color según valor
        color = obtener_color_por_valor(row['valor_en_uf'])
        
        # Crear el popup
        popup_content = crear_popup_info(row)
        
        # Crear el ícono con color directo
        icon_html = f"""
        <div style="
            text-align: center; 
            line-height: 30px;
            width: 30px;
            height: 30px;
        ">
            <i class="fa {icono}" style="
                font-size: 20px;
                color: {color};
                text-shadow: 1px 1px 3px rgba(0,0,0,0.7);
            "></i>
        </div>
        """
        
        custom_icon = folium.DivIcon(
            html=icon_html,
            icon_size=(30, 30),
            icon_anchor=(15, 15)
        )
        
        # Agregar Marker
        folium.Marker(
            location=[row['latitud'], row['longitud']],
            popup=folium.Popup(popup_content, max_width=270),
            tooltip=f"{row['tipo_propiedad'].title() if pd.notna(row['tipo_propiedad']) else 'Propiedad'} - {row['valor_en_uf']:,.0f} UF" if pd.notna(row['valor_en_uf']) else f"{row['tipo_propiedad'].title() if pd.notna(row['tipo_propiedad']) else 'Propiedad'}",
            icon=custom_icon
        ).add_to(mapa)

# leyenda
leyenda_html = f"""
<div style="position: fixed; 
            top: 10px; right: 10px; width: 220px; height: auto; 
            background-color: white; border:2px solid grey; z-index:9999; 
            font-size:11px; padding: 10px; border-radius: 5px;">

<div style="margin-bottom: 15px;">
<h5 style="margin: 5px 0; color: #333;">Tipos de Propiedad:</h5>
<p style="margin: 3px 0; display: flex; align-items: center;">
    <i class="fa fa-home" style="font-size: 14px; margin-right: 8px; color: #666;"></i> Casa
</p>
<p style="margin: 3px 0; display: flex; align-items: center;">
    <i class="fa fa-building" style="font-size: 14px; margin-right: 8px; color: #666;"></i> Departamento
</p>
<p style="margin: 3px 0; display: flex; align-items: center;">
    <i class="fa fa-tree" style="font-size: 14px; margin-right: 8px; color: #666;"></i> Parcela
</p>
<p style="margin: 3px 0; display: flex; align-items: center;">
    <i class="fa fa-briefcase" style="font-size: 14px; margin-right: 8px; color: #666;"></i> Oficina
</p>
<p style="margin: 3px 0; display: flex; align-items: center;">
    <i class="fa fa-shopping-cart" style="font-size: 14px; margin-right: 8px; color: #666;"></i> Local
</p>
<p style="margin: 3px 0; display: flex; align-items: center;">
    <i class="fa fa-map" style="font-size: 14px; margin-right: 8px; color: #666;"></i> Terreno
</p>
<p style="margin: 3px 0; display: flex; align-items: center;">
    <i class="fa fa-archive" style="font-size: 14px; margin-right: 8px; color: #666;"></i> Bodega
</p>
</div>

<div>
<h5 style="margin: 5px 0; color: #333;">Rangos de Precio (UF):</h5>
<p style="margin: 2px 0;"><span style="color: #00B050; font-size: 18px; font-weight: bold;">●</span> Hasta 1,000 UF</p>
<p style="margin: 2px 0;"><span style="color: #92D050; font-size: 18px; font-weight: bold;">●</span> 1,001 - 2,500 UF</p>
<p style="margin: 2px 0;"><span style="color: #FF8C00; font-size: 18px; font-weight: bold;">●</span> 2,501 - 5,000 UF</p>
<p style="margin: 2px 0;"><span style="color: #FF4500; font-size: 18px; font-weight: bold;">●</span> 5,001 - 10,000 UF</p>
<p style="margin: 2px 0;"><span style="color: #B22222; font-size: 18px; font-weight: bold;">●</span> Más de 10,000 UF</p>
<p style="margin: 2px 0;"><span style="color: #808080; font-size: 18px; font-weight: bold;">●</span> Sin precio</p>
</div>

</div>
"""

mapa.get_root().html.add_child(folium.Element(leyenda_html))

# Guardar mapa
mapa.save('mapa_propiedades.html')